In [ ]:
# @title 🔑 Einmalig Global Einloggen (Fix)
# ==============================================================================
# Führe dies EINMAL aus. Danach ist 'creds' global verfügbar.
# Alle nachfolgenden Skripte greifen darauf zu.
# ==============================================================================

# Wir nutzen die Auth-Funktion aus dem allerersten Setup-Block
try:
    # Speichert die Credentials global in der Variable 'creds'
    creds = authenticate_analytics()
    print("\n✅ Erfolgreich! Die Anmeldung ist nun global gespeichert.")
    print("👉 Du kannst jetzt alle Analyse-Skripte ausführen, ohne dich neu einzuloggen.")
except NameError:
    print("❌ Fehler: Bitte führe zuerst ganz oben den Block 1 (Setup & Imports) aus, damit die Funktion 'authenticate_analytics' bekannt ist.")

In [ ]:
# @title 📈 2025 Verhältnis-Analyse: Organic (Basis 100) vs. LLM
# ==============================================================================
# LOGIK-ÄNDERUNG:
# Index 100 = Organischer Traffic im Januar.
# Die LLM-Kurve zeigt das Volumen RELATIV zum organischen Startwert.
# (Beispiel: Index 1 bei LLM bedeutet, LLM hat 1% des organischen Volumens)
# ==============================================================================

import re
import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from googleapiclient.discovery import build

# --- KONFIGURATION ---
START_DATE = "2025-01-01"
END_DATE   = "2025-12-31"

LLM_REGEX = r"^.*ai|.*\.openai.*|.*copilot.*|.*chatgpt.*|.*gemini.*|.*gpt.*|.*neeva.*|.*writesonic.*|.*nimble.*|.*outrider.*|.*perplexity.*|.*google.*bard.*|.*bard.*google.*|.*bard.*|.*edgeservices.*|.*astastic.*|.*copy\.ai.*|.*bnngpt.*|.*gemini.*google.*$"

def get_creds_smart():
    if 'creds' in globals() and globals()['creds'] and globals()['creds'].valid:
        return globals()['creds']
    try:
        return authenticate_analytics()
    except NameError:
        print("⚠️ Bitte Block 1 (Auth) zuerst ausführen.")
        return None

def classify_traffic_source(source, medium):
    s = str(source).lower()
    m = str(medium).lower()
    if re.match(LLM_REGEX, s): return "LLM / AI"
    if 'organic' in m: return "Organic Search"
    return "Other"

def run_relation_analysis_2025():
    creds = get_creds_smart()
    if not creds: return

    print(f"\n🚀 Berechne Verhältnis Organic vs. LLM (Basis: Organic Jan = 100)...")

    # Services & Properties
    try:
        admin_service = build('analyticsadmin', 'v1beta', credentials=creds)
        data_service = build('analyticsdata', 'v1beta', credentials=creds)
        acc_summary = admin_service.accountSummaries().list().execute()
        all_props = [p['property'].split('/')[1] for acc in acc_summary.get('accountSummaries', []) for p in acc.get('propertySummaries', [])]
        print(f"👉 Analysiere {len(all_props)} Properties...")
    except Exception as e:
        print(f"❌ Fehler: {e}")
        return

    # Datenabruf
    daily_aggs = []
    for p_id in tqdm(all_props, desc="Lade Daten"):
        try:
            req = {
                "dateRanges": [{"startDate": START_DATE, "endDate": END_DATE}],
                "dimensions": [{"name": "yearMonth"}, {"name": "sessionSource"}, {"name": "sessionMedium"}],
                "metrics": [{"name": "sessions"}]
            }
            res = data_service.properties().runReport(property=f"properties/{p_id}", body=req).execute()
            if 'rows' not in res: continue

            rows = []
            for r in res['rows']:
                rows.append({
                    'Month': r['dimensionValues'][0]['value'],
                    'Source': r['dimensionValues'][1]['value'],
                    'Medium': r['dimensionValues'][2]['value'],
                    'Sessions': int(r['metricValues'][0]['value'])
                })
            df = pd.DataFrame(rows)
            df['Channel'] = df.apply(lambda x: classify_traffic_source(x['Source'], x['Medium']), axis=1)
            daily_aggs.append(df.groupby(['Month', 'Channel'])['Sessions'].sum().reset_index())
            time.sleep(0.05)
        except: continue

    if not daily_aggs:
        print("❌ Keine Daten.")
        return

    # Aggregation
    full_df = pd.concat(daily_aggs)
    total_trend = full_df.groupby(['Month', 'Channel'])['Sessions'].sum().reset_index()
    relevant = total_trend[total_trend['Channel'].isin(['Organic Search', 'LLM / AI'])].copy()
    pivot = relevant.pivot(index='Month', columns='Channel', values='Sessions').fillna(0)

    # --- NEUE INDEX LOGIK ---
    pivot_idx = pd.DataFrame(index=pivot.index)

    try:
        # 1. Wir holen uns den absoluten Wert von Organic im Januar (oder ersten Monat)
        # Das ist unser universeller Teiler für ALLES.
        base_volume = pivot['Organic Search'].iloc[0]

        if base_volume == 0:
            print("❌ Fehler: Organischer Traffic im Startmonat ist 0. Kann keine Basis bilden.")
            return

        print(f"\nℹ️ BASIS-WERT (Index 100): {base_volume:,.0f} Organische Sessions im Startmonat.")

        # 2. Organic Index berechnen (Startet bei 100)
        pivot_idx['Organic Search'] = (pivot['Organic Search'] / base_volume) * 100

        # 3. LLM Index berechnen (Startet bei X, relativ zu Organic)
        # Wenn LLM 1% von Organic hat, steht hier 1.
        pivot_idx['LLM / AI'] = (pivot['LLM / AI'] / base_volume) * 100

    except Exception as e:
        print(f"❌ Fehler bei Berechnung: {e}")
        return

    print("\n" + "="*60)
    print("📈 INDEX-TABELLE (Organic Jan = 100)")
    print("="*60)
    print("Interpretation: Ein LLM-Wert von '2.5' bedeutet, dass LLM-Traffic")
    print("2.5% des Volumens vom organischen Traffic im Januar entspricht.")
    print("-" * 60)
    print(pivot_idx.round(2).to_markdown())

    # --- PLOTTING ---
    plt.figure(figsize=(12, 6))

    # Organic Plot
    sns.lineplot(data=pivot_idx, x=pivot_idx.index, y='Organic Search',
                 color='lightgray', linewidth=2, label='Organic Search (Basis)')

    # LLM Plot
    sns.lineplot(data=pivot_idx, x=pivot_idx.index, y='LLM / AI',
                 color='#a64eff', linewidth=4, marker='o', markersize=8, label='LLM / AI (Relativ)')

    # Beschriftung
    plt.axhline(100, color='black', linestyle=':', alpha=0.3, label='Baseline (100)')
    plt.title('Traffic-Verhältnis 2025: Wie gross ist LLM im Vergleich zu Organic?', fontsize=16)
    plt.ylabel('Index (Organic Jan = 100)', fontsize=12)
    plt.xlabel('Monat')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.2)
    plt.legend()
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    run_relation_analysis_2025()

In [ ]:
# @title 🤖 LLM Solo-Analyse 2025 (Index 100 = Start)
# ==============================================================================
# Zeigt NUR die Entwicklung des LLM/AI Traffics.
# Der erste Monat (Januar) wird auf 100 gesetzt.
# Farbe: #a64eff
# ==============================================================================

import re
import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from googleapiclient.discovery import build

# --- KONFIGURATION ---
START_DATE = "2025-01-01"
END_DATE   = "2025-12-31"

# Dein spezifischer Regex
LLM_REGEX = r"^.*ai|.*\.openai.*|.*copilot.*|.*chatgpt.*|.*gemini.*|.*gpt.*|.*neeva.*|.*writesonic.*|.*nimble.*|.*outrider.*|.*perplexity.*|.*google.*bard.*|.*bard.*google.*|.*bard.*|.*edgeservices.*|.*astastic.*|.*copy\.ai.*|.*bnngpt.*|.*gemini.*google.*$"

def get_creds_smart():
    if 'creds' in globals() and globals()['creds'] and globals()['creds'].valid:
        return globals()['creds']
    try:
        return authenticate_analytics()
    except NameError:
        print("⚠️ Bitte Block 1 (Auth) zuerst ausführen.")
        return None

def is_llm_traffic(source):
    """Prüft nur auf den LLM Regex."""
    return bool(re.match(LLM_REGEX, str(source).lower()))

def run_llm_solo_analysis():
    creds = get_creds_smart()
    if not creds: return

    print(f"\n🚀 Starte isolierte LLM-Analyse 2025...")

    # Setup
    try:
        admin_service = build('analyticsadmin', 'v1beta', credentials=creds)
        data_service = build('analyticsdata', 'v1beta', credentials=creds)

        # Properties laden
        acc_summary = admin_service.accountSummaries().list().execute()
        all_props = []
        for acc in acc_summary.get('accountSummaries', []):
            for p in acc.get('propertySummaries', []):
                all_props.append(p['property'].split('/')[1])

        print(f"👉 Scanne {len(all_props)} Properties nach AI-Traffic...")

    except Exception as e:
        print(f"❌ Fehler: {e}")
        return

    # Datenabruf
    monthly_llm_sessions = []

    for p_id in tqdm(all_props, desc="Suche AI Traffic"):
        try:
            req = {
                "dateRanges": [{"startDate": START_DATE, "endDate": END_DATE}],
                "dimensions": [{"name": "yearMonth"}, {"name": "sessionSource"}], # Medium brauchen wir hier nicht zwingend
                "metrics": [{"name": "sessions"}]
            }
            res = data_service.properties().runReport(property=f"properties/{p_id}", body=req).execute()

            if 'rows' not in res: continue

            for row in res['rows']:
                source = row['dimensionValues'][1]['value']

                # Filter greift direkt hier
                if is_llm_traffic(source):
                    month = row['dimensionValues'][0]['value']
                    sessions = int(row['metricValues'][0]['value'])

                    monthly_llm_sessions.append({'Month': month, 'Sessions': sessions})

        except:
            continue

    if not monthly_llm_sessions:
        print("❌ Keinen einzigen Session mit passendem LLM-Source gefunden.")
        return

    # --- AGGREGATION ---
    df = pd.DataFrame(monthly_llm_sessions)

    # Summe aller LLM Sessions pro Monat über alle Properties
    trend = df.groupby('Month')['Sessions'].sum().reset_index().sort_values('Month')

    # --- INDEX BERECHNUNG ---
    # Basis: Erster Monat = 100
    try:
        base_val = trend.iloc[0]['Sessions']
        start_month = trend.iloc[0]['Month']

        if base_val == 0:
            # Suche ersten Monat > 0
            first_valid = trend[trend['Sessions'] > 0].iloc[0]
            base_val = first_valid['Sessions']
            start_month = first_valid['Month']
            print(f"ℹ️ Startmonat hatte 0 Traffic. Basis verschoben auf: {start_month}")

        trend['LLM Index'] = (trend['Sessions'] / base_val) * 100

        print("\n" + "="*60)
        print(f"🤖 LLM WACHSTUM 2025 (Basis: {start_month} = 100)")
        print("="*60)
        print(trend[['Month', 'Sessions', 'LLM Index']].round(1).to_markdown(index=False))

        # --- PLOT ---
        plt.figure(figsize=(10, 6))

        sns.lineplot(data=trend, x='Month', y='LLM Index',
                     color='#a64eff', linewidth=4, marker='o', markersize=9)

        plt.axhline(100, color='gray', linestyle=':', alpha=0.5)
        plt.title('Wachstum LLM / AI Traffic (Isoliert)', fontsize=16)
        plt.ylabel('Index (Start = 100)', fontsize=12)
        plt.xlabel('Monat 2025')
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.2)
        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"❌ Fehler bei der Berechnung: {e}")

if __name__ == "__main__":
    run_llm_solo_analysis()

In [ ]:
# @title 📅 Qualitäts-Benchmark: LLM ist die Basis (100)
# ==============================================================================
# LOGIK NEU:
# Wir berechnen den Wert für JEDEN Monat neu relativ zu LLM.
# Formel: (Wert Organic Monat X / Wert LLM Monat X) * 100
#
# ERGEBNIS:
# Die LLM-Linie wird eine flache Linie bei 100 sein.
# Die Organic-Linie zeigt, wie viel % besser/schlechter sie im Vergleich ist.
# ==============================================================================

import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import re
import numpy as np
from tqdm import tqdm
from googleapiclient.discovery import build

# --- KONFIGURATION ---
START_DATE = "2025-01-01"
END_DATE   = "2025-12-31"
MIN_SESSIONS_MONTHLY = 20

# --- GLOBALE DEFINITIONEN ---
LLM_REGEX = r"^.*ai|.*\.openai.*|.*copilot.*|.*chatgpt.*|.*gemini.*|.*gpt.*|.*neeva.*|.*writesonic.*|.*nimble.*|.*outrider.*|.*perplexity.*|.*google.*bard.*|.*bard.*google.*|.*bard.*|.*edgeservices.*|.*astastic.*|.*copy\.ai.*|.*bnngpt.*|.*gemini.*google.*$"
SEARCH_ENGINES = ['google', 'bing', 'yahoo', 'duckduckgo', 'ecosia', 'startpage', 'baidu', 'yandex']
SOCIAL_SOURCES = ['facebook', 'instagram', 'linkedin', 'twitter', 't.co', 'tiktok', 'pinterest', 'reddit', 'youtube']

def get_creds_smart():
    if 'creds' in globals() and globals()['creds'] and globals()['creds'].valid:
        return globals()['creds']
    try:
        return authenticate_analytics()
    except NameError:
        print("⚠️ Bitte Block 1 (Auth) zuerst ausführen.")
        return None

# --- EINHEITLICHE KLASSIFIZIERUNG FÜR BEIDE SCRIPTS ---
def classify_universal(source, medium):
    s = str(source).lower()
    m = str(medium).lower()

    # 1. LLM (Priorität 1)
    if re.match(LLM_REGEX, s):
        return "LLM / AI"

    # 2. Google Ads (Priorität 2)
    if 'google' in s and ('cpc' in m or 'ppc' in m or 'paid' in m):
        return "Google Ads"

    # 3. Social (Priorität 3) - Wichtig: Vor Organic, damit 'insta/organic' nicht falsch zählt
    if 'social' in m or any(soc in s for soc in SOCIAL_SOURCES):
        return "Social"

    # 4. Organic Search (Priorität 4)
    if 'organic' in m or (m == 'referral' and any(se in s for se in SEARCH_ENGINES)):
        return "Organic Search"

    # 5. Direct
    if '(direct)' in s:
        return "Direct"

    return "Other"

def get_data_2025(service, property_id):
    metrics_try = [{"name": "sessions"}, {"name": "purchaseRevenue"}, {"name": "keyEvents"}]
    req_body = {
        "dateRanges": [{"startDate": START_DATE, "endDate": END_DATE}],
        "dimensions": [{"name": "yearMonth"}, {"name": "sessionSource"}, {"name": "sessionMedium"}],
        "metrics": metrics_try
    }
    try:
        return service.properties().runReport(property=f"properties/{property_id}", body=req_body).execute()
    except: return None

def run_benchmark_focused_flat():
    creds = get_creds_smart()
    if not creds: return

    print(f"\n🔍 Starte Benchmark (LLM = 100 Basislinie)...")

    try:
        admin_service = build('analyticsadmin', 'v1beta', credentials=creds)
        data_service = build('analyticsdata', 'v1beta', credentials=creds)
        acc_summary = admin_service.accountSummaries().list().execute()
        all_props = [{'id': p['property'].split('/')[1], 'name': p.get('displayName')}
                     for acc in acc_summary.get('accountSummaries', [])
                     for p in acc.get('propertySummaries', [])]
    except: return

    collected_data = []

    for prop in tqdm(all_props, desc="Analysiere Properties"):
        res = get_data_2025(data_service, prop['id'])
        if not res or 'rows' not in res: continue

        data = []
        for row in res['rows']:
            # Metriken sicher extrahieren
            try: rev = float(row['metricValues'][1]['value'])
            except: rev = 0.0
            try: conv = float(row['metricValues'][2]['value'])
            except: conv = 0.0

            data.append({
                'MonthStr': row['dimensionValues'][0]['value'],
                'Source': row['dimensionValues'][1]['value'],
                'Medium': row['dimensionValues'][2]['value'],
                'Sessions': int(row['metricValues'][0]['value']),
                'Revenue': rev,
                'Conversions': conv
            })

        df = pd.DataFrame(data)
        if df.empty or df['Sessions'].sum() < 50: continue

        df['Date'] = pd.to_datetime(df['MonthStr'], format='%Y%m')

        # Klassifizierung
        df['Channel'] = df.apply(lambda x: classify_universal(x['Source'], x['Medium']), axis=1)

        # Shop-Entscheidung auf Gesamt-Daten
        is_shop = df['Revenue'].sum() > 100

        # Aggregation pro Monat
        monthly = df.groupby(['Date', 'Channel']).agg({
            'Sessions': 'sum', 'Revenue': 'sum', 'Conversions': 'sum'
        }).reset_index()

        # --- BENCHMARK BERECHNUNG (Monat für Monat) ---
        # Wir iterieren durch jeden vorhandenen Monat dieser Property
        for month_date in monthly['Date'].unique():
            month_data = monthly[monthly['Date'] == month_date]

            # 1. Hole den LLM-Wert für diesen Monat (Basis)
            llm_row = month_data[month_data['Channel'] == 'LLM / AI']

            # Wenn kein LLM Traffic in diesem Monat -> Monat überspringen (kein Vergleich möglich)
            if llm_row.empty or llm_row['Sessions'].sum() < MIN_SESSIONS_MONTHLY:
                continue

            llm_sessions = llm_row['Sessions'].sum()
            if is_shop:
                llm_val = llm_row['Revenue'].sum() / llm_sessions
            else:
                llm_val = (llm_row['Conversions'].sum() / llm_sessions) * 100

            if llm_val <= 0: continue

            # 2. Berechne Index für Organic (und setze LLM auf 100)
            # Wir schauen uns hier nur Organic und LLM an
            for channel in ['LLM / AI', 'Organic Search']:
                ch_row = month_data[month_data['Channel'] == channel]
                if ch_row.empty or ch_row['Sessions'].sum() < MIN_SESSIONS_MONTHLY:
                    continue

                if is_shop:
                    val = ch_row['Revenue'].sum() / ch_row['Sessions'].sum()
                else:
                    val = (ch_row['Conversions'].sum() / ch_row['Sessions'].sum()) * 100

                # DER BENCHMARK:
                index = (val / llm_val) * 100

                collected_data.append({
                    'Date': month_date,
                    'Index': index,
                    'Channel': channel
                })

        time.sleep(0.01)

    # --- AUSWERTUNG ---
    df_all = pd.DataFrame(collected_data)
    if df_all.empty:
        print("\n❌ Keine vergleichbaren Daten (LLM vs Organic) gefunden.")
        return

    # Median berechnen
    final_trend = df_all.groupby(['Date', 'Channel'])['Index'].median().reset_index()

    # Tabelle
    pivot = final_trend.pivot(index='Date', columns='Channel', values='Index')
    pivot.index = pivot.index.strftime('%Y-%m')

    print("\n" + "="*70)
    print(f"📊 QUALITÄTS-BENCHMARK (LLM ist immer 100)")
    print(f"Interpretation: 150 = Organic ist 50% wertvoller als AI in diesem Monat")
    print("="*70)
    print(pivot.round(1).to_markdown())

    # Grafik
    plt.figure(figsize=(12, 6))

    # LLM (Referenz)
    data_ai = final_trend[final_trend['Channel']=='LLM / AI']
    sns.lineplot(data=data_ai, x='Date', y='Index',
                 label='LLM / AI (Benchmark)', color='#a64eff', linewidth=4, linestyle='-')

    # Organic
    data_org = final_trend[final_trend['Channel']=='Organic Search']
    if not data_org.empty:
        sns.lineplot(data=data_org, x='Date', y='Index',
                     label='Organic Search', color='#0F9D58', linewidth=3, marker='o')

        # Label
        last_org = data_org.iloc[-1]
        plt.text(last_org['Date'], last_org['Index'] + 5, f"{last_org['Index']:.0f}",
                 color='#0F9D58', fontweight='bold', ha='center', va='bottom')

    plt.title('Ist Organic besser als AI? (Benchmark = 100)', fontsize=14)
    plt.ylabel('Qualitäts-Index (LLM = 100)')
    plt.xlabel('')

    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.grid(True, alpha=0.2)
    plt.legend()
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    run_benchmark_focused_flat()

In [ ]:
# @title 📊 5-Kanal Benchmark: LLM ist die Basis (100)
# ==============================================================================
# Vergleicht 5 Kanäle.
# LLM ist immer 100.
# Organic Zahlen sind EXAKT gleich wie im ersten Script.
# ==============================================================================

import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import re
import numpy as np
from tqdm import tqdm
from googleapiclient.discovery import build

# --- KONFIGURATION ---
START_DATE = "2025-01-01"
END_DATE   = "2025-12-31"
MIN_SESSIONS_MONTHLY = 20

# --- GLOBALE DEFINITIONEN (IDENTISCH) ---
LLM_REGEX = r"^.*ai|.*\.openai.*|.*copilot.*|.*chatgpt.*|.*gemini.*|.*gpt.*|.*neeva.*|.*writesonic.*|.*nimble.*|.*outrider.*|.*perplexity.*|.*google.*bard.*|.*bard.*google.*|.*bard.*|.*edgeservices.*|.*astastic.*|.*copy\.ai.*|.*bnngpt.*|.*gemini.*google.*$"
SEARCH_ENGINES = ['google', 'bing', 'yahoo', 'duckduckgo', 'ecosia', 'startpage', 'baidu', 'yandex']
SOCIAL_SOURCES = ['facebook', 'instagram', 'linkedin', 'twitter', 't.co', 'tiktok', 'pinterest', 'reddit', 'youtube']

def get_creds_smart():
    if 'creds' in globals() and globals()['creds'] and globals()['creds'].valid:
        return globals()['creds']
    try:
        return authenticate_analytics()
    except NameError:
        print("⚠️ Bitte Block 1 (Auth) zuerst ausführen.")
        return None

# --- EINHEITLICHE KLASSIFIZIERUNG (1:1 KOPIE VON SCRIPT 1) ---
def classify_universal(source, medium):
    s = str(source).lower()
    m = str(medium).lower()

    # 1. LLM
    if re.match(LLM_REGEX, s): return "LLM / AI"
    # 2. Ads
    if 'google' in s and ('cpc' in m or 'ppc' in m or 'paid' in m): return "Google Ads"
    # 3. Social
    if 'social' in m or any(soc in s for soc in SOCIAL_SOURCES): return "Social"
    # 4. Organic
    if 'organic' in m or (m == 'referral' and any(se in s for se in SEARCH_ENGINES)): return "Organic Search"
    # 5. Direct
    if '(direct)' in s: return "Direct"

    return "Other"

def get_data_2025(service, property_id):
    metrics_try = [{"name": "sessions"}, {"name": "purchaseRevenue"}, {"name": "keyEvents"}]
    req_body = {
        "dateRanges": [{"startDate": START_DATE, "endDate": END_DATE}],
        "dimensions": [{"name": "yearMonth"}, {"name": "sessionSource"}, {"name": "sessionMedium"}],
        "metrics": metrics_try
    }
    try:
        return service.properties().runReport(property=f"properties/{property_id}", body=req_body).execute()
    except: return None

def run_5_channel_benchmark_flat():
    creds = get_creds_smart()
    if not creds: return

    print(f"\n🔍 Starte 5-Kanal Benchmark (Synchronisiert)...")

    try:
        admin_service = build('analyticsadmin', 'v1beta', credentials=creds)
        data_service = build('analyticsdata', 'v1beta', credentials=creds)
        acc_summary = admin_service.accountSummaries().list().execute()
        all_props = [{'id': p['property'].split('/')[1], 'name': p.get('displayName')}
                     for acc in acc_summary.get('accountSummaries', [])
                     for p in acc.get('propertySummaries', [])]
    except: return

    collected_data = []

    for prop in tqdm(all_props, desc="Verarbeite Properties"):
        res = get_data_2025(data_service, prop['id'])
        if not res or 'rows' not in res: continue

        data = []
        for row in res['rows']:
            try: rev = float(row['metricValues'][1]['value'])
            except: rev = 0.0
            try: conv = float(row['metricValues'][2]['value'])
            except: conv = 0.0

            data.append({
                'MonthStr': row['dimensionValues'][0]['value'],
                'Source': row['dimensionValues'][1]['value'],
                'Medium': row['dimensionValues'][2]['value'],
                'Sessions': int(row['metricValues'][0]['value']),
                'Revenue': rev,
                'Conversions': conv
            })

        df = pd.DataFrame(data)
        if df.empty or df['Sessions'].sum() < 50: continue

        df['Date'] = pd.to_datetime(df['MonthStr'], format='%Y%m')

        # Klassifizierung
        df['Channel'] = df.apply(lambda x: classify_universal(x['Source'], x['Medium']), axis=1)

        # Shop Entscheidung (global)
        is_shop = df['Revenue'].sum() > 100

        # Aggregation
        monthly = df.groupby(['Date', 'Channel']).agg({
            'Sessions': 'sum', 'Revenue': 'sum', 'Conversions': 'sum'
        }).reset_index()

        # --- BENCHMARK PRO MONAT ---
        relevant_channels = ['LLM / AI', 'Google Ads', 'Social', 'Organic Search', 'Direct']

        for month_date in monthly['Date'].unique():
            month_data = monthly[monthly['Date'] == month_date]

            # 1. LLM Basis für diesen Monat holen
            llm_row = month_data[month_data['Channel'] == 'LLM / AI']

            if llm_row.empty or llm_row['Sessions'].sum() < MIN_SESSIONS_MONTHLY:
                continue

            llm_sessions = llm_row['Sessions'].sum()
            if is_shop:
                llm_val = llm_row['Revenue'].sum() / llm_sessions
            else:
                llm_val = (llm_row['Conversions'].sum() / llm_sessions) * 100

            if llm_val <= 0: continue

            # 2. Andere Kanäle dagegen messen
            for channel in relevant_channels:
                ch_row = month_data[month_data['Channel'] == channel]
                if ch_row.empty or ch_row['Sessions'].sum() < MIN_SESSIONS_MONTHLY:
                    continue

                if is_shop:
                    val = ch_row['Revenue'].sum() / ch_row['Sessions'].sum()
                else:
                    val = (ch_row['Conversions'].sum() / ch_row['Sessions'].sum()) * 100

                # INDEX
                index = (val / llm_val) * 100

                collected_data.append({
                    'Date': month_date,
                    'Index': index,
                    'Channel': channel
                })

        time.sleep(0.01)

    # --- AUSWERTUNG ---
    if not collected_data:
        print("\n❌ Keine Daten.")
        return

    df_all = pd.DataFrame(collected_data)

    # Median Aggregation
    final_trend = df_all.groupby(['Date', 'Channel'])['Index'].median().reset_index()

    # Tabelle
    pivot = final_trend.pivot(index='Date', columns='Channel', values='Index')
    pivot.index = pivot.index.strftime('%Y-%m')

    desired_order = ['LLM / AI', 'Organic Search', 'Google Ads', 'Social', 'Direct']
    cols = [c for c in desired_order if c in pivot.columns]
    pivot = pivot[cols]

    print("\n" + "="*70)
    print(f"📊 5-KANAL BENCHMARK (LLM = 100 Basis)")
    print("="*70)
    print(pivot.round(1).to_markdown())

    # --- PLOTTING ---
    plt.figure(figsize=(14, 8))

    colors = {
        'LLM / AI': '#a64eff',       # Violett
        'Google Ads': '#DB4437',     # Rot
        'Social': '#4285F4',         # Blau
        'Organic Search': '#0F9D58', # Grün
        'Direct': '#757575'          # Grau
    }

    for channel in cols:
        subset = final_trend[final_trend['Channel'] == channel]

        lw = 4 if channel == 'LLM / AI' else (3 if channel == 'Organic Search' else 1.5)
        style = '-' # Alle solid, LLM dicker
        alpha = 1.0 if channel in ['LLM / AI', 'Organic Search'] else 0.7

        sns.lineplot(data=subset, x='Date', y='Index',
                     label=channel, color=colors.get(channel, 'black'),
                     linewidth=lw, linestyle=style, marker='o', alpha=alpha)

        if not subset.empty:
            last = subset.iloc[-1]
            plt.text(last['Date'], last['Index'], f" {last['Index']:.0f}",
                     color=colors.get(channel, 'black'), fontweight='bold', va='center')

    plt.axhline(100, color='black', linestyle=':', alpha=0.4)
    plt.title('Qualitäts-Vergleich: Wer schlägt den AI-Traffic?', fontsize=16)
    plt.ylabel('Qualitäts-Index (LLM = 100)', fontsize=12)
    plt.xlabel('')

    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.grid(True, alpha=0.2)
    plt.legend(loc='upper left')
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    run_5_channel_benchmark_flat()

In [ ]:
# @title 💰 Conversion Rate Analyse 2025: Organic vs. LLM
# ==============================================================================
# Vergleicht die Qualität des Traffics (KeyEvents / Sessions * 100)
# Farbe LLM: #a64eff
# Zeitraum: Jan - Dez 2025
# ==============================================================================

import re
import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from tqdm import tqdm
from googleapiclient.discovery import build

# --- KONFIGURATION ---
START_DATE = "2025-01-01"
END_DATE   = "2025-12-31"

# Dein spezifischer Regex
LLM_REGEX = r"^.*ai|.*\.openai.*|.*copilot.*|.*chatgpt.*|.*gemini.*|.*gpt.*|.*neeva.*|.*writesonic.*|.*nimble.*|.*outrider.*|.*perplexity.*|.*google.*bard.*|.*bard.*google.*|.*bard.*|.*edgeservices.*|.*astastic.*|.*copy\.ai.*|.*bnngpt.*|.*gemini.*google.*$"

def get_creds_smart():
    if 'creds' in globals() and globals()['creds'] and globals()['creds'].valid:
        return globals()['creds']
    try:
        return authenticate_analytics()
    except NameError:
        print("⚠️ Bitte Block 1 (Auth) zuerst ausführen.")
        return None

def classify_traffic_source(source, medium):
    s = str(source).lower()
    m = str(medium).lower()

    # 1. Regex Check (Priorität)
    if re.match(LLM_REGEX, s):
        return "LLM / AI"

    # 2. Organic Search
    if 'organic' in m:
        return "Organic Search"

    return "Other"

def get_conversion_data(service, property_id):
    """Holt Sessions UND KeyEvents (mit Fallback auf Conversions)."""
    # Wir brauchen beides: Sessions (Nenner) und KeyEvents (Zähler)
    metrics_try = [{"name": "sessions"}, {"name": "keyEvents"}]

    req = {
        "dateRanges": [{"startDate": START_DATE, "endDate": END_DATE}],
        "dimensions": [{"name": "yearMonth"}, {"name": "sessionSource"}, {"name": "sessionMedium"}],
        "metrics": metrics_try
    }

    try:
        return service.properties().runReport(property=f"properties/{property_id}", body=req).execute()
    except Exception as e:
        # Fallback für alte Properties (legacy 'conversions')
        if "keyEvents" in str(e) or "400" in str(e):
            req['metrics'] = [{"name": "sessions"}, {"name": "conversions"}]
            try:
                return service.properties().runReport(property=f"properties/{property_id}", body=req).execute()
            except:
                return None
        return None

def run_cvr_analysis_2025():
    creds = get_creds_smart()
    if not creds: return

    print(f"\n🚀 Berechne Conversion Rate Vergleich (2025)...")

    # Services
    try:
        admin_service = build('analyticsadmin', 'v1beta', credentials=creds)
        data_service = build('analyticsdata', 'v1beta', credentials=creds)

        acc_summary = admin_service.accountSummaries().list().execute()
        all_props = []
        for acc in acc_summary.get('accountSummaries', []):
            for p in acc.get('propertySummaries', []):
                all_props.append(p['property'].split('/')[1])

        print(f"👉 Analysiere Qualität von {len(all_props)} Properties...")

    except Exception as e:
        print(f"❌ Fehler: {e}")
        return

    # Daten sammeln
    raw_data = []

    for p_id in tqdm(all_props, desc="Lade Daten"):
        res = get_conversion_data(data_service, p_id)

        if not res or 'rows' not in res: continue

        for r in res['rows']:
            sessions = int(r['metricValues'][0]['value'])
            conversions = float(r['metricValues'][1]['value'])

            # Nur Daten speichern, wenn Traffic da ist
            if sessions > 0:
                raw_data.append({
                    'Month': r['dimensionValues'][0]['value'],
                    'Source': r['dimensionValues'][1]['value'],
                    'Medium': r['dimensionValues'][2]['value'],
                    'Sessions': sessions,
                    'Conversions': conversions
                })

        time.sleep(0.05)

    if not raw_data:
        print("❌ Keine Daten verfügbar.")
        return

    # --- AGGREGATION ---
    df = pd.DataFrame(raw_data)

    # Klassifizieren
    df['Channel'] = df.apply(lambda x: classify_traffic_source(x['Source'], x['Medium']), axis=1)

    # Wir filtern nur Organic und LLM
    df = df[df['Channel'].isin(['Organic Search', 'LLM / AI'])]

    # Gruppieren über ALLE Properties (Portfolio View)
    # Summe Conversions / Summe Sessions = Wahre Portfolio CVR
    grouped = df.groupby(['Month', 'Channel']).agg({
        'Sessions': 'sum',
        'Conversions': 'sum'
    }).reset_index()

    # CVR Berechnen
    grouped['CVR %'] = (grouped['Conversions'] / grouped['Sessions']) * 100

    # Pivot für Tabelle
    pivot = grouped.pivot(index='Month', columns='Channel', values='CVR %').fillna(0)

    print("\n" + "="*60)
    print("💰 CONVERSION RATE (%) IM VERGLEICH")
    print("="*60)
    print(pivot.round(2).to_markdown())

    # --- PLOTTING ---
    plt.figure(figsize=(12, 6))

    # Organic Linie
    sns.lineplot(data=grouped[grouped['Channel']=='Organic Search'], x='Month', y='CVR %',
                 color='gray', linewidth=2, label='Organic Search (CVR)')

    # LLM Linie
    sns.lineplot(data=grouped[grouped['Channel']=='LLM / AI'], x='Month', y='CVR %',
                 color='#a64eff', linewidth=4, marker='o', markersize=8, label='LLM / AI (CVR)')

    plt.title('Qualitäts-Vergleich 2025: Conversion Rate (Key Events / Sessions)', fontsize=15)
    plt.ylabel('Conversion Rate (%)', fontsize=12)
    plt.xlabel('Monat')

    # Y-Achse als Prozent formatieren
    plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())

    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.2)
    plt.legend()
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    run_cvr_analysis_2025()

In [ ]:
# @title 💰 CVR Analyse 2025: Median pro Property (Fairer Vergleich)
# ==============================================================================
# Berechnet CVR pro Property und bildet dann den Median.
# Verhindert, dass grosse Accounts den Durchschnitt verzerren.
# Farbe LLM: #a64eff
# ==============================================================================

import re
import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from tqdm import tqdm
from googleapiclient.discovery import build

# --- KONFIGURATION ---
START_DATE = "2025-01-01"
END_DATE   = "2025-12-31"
MIN_SESSIONS = 50 # Mindest-Traffic pro Monat/Property, um 0% CVRs durch Rauschen zu filtern

LLM_REGEX = r"^.*ai|.*\.openai.*|.*copilot.*|.*chatgpt.*|.*gemini.*|.*gpt.*|.*neeva.*|.*writesonic.*|.*nimble.*|.*outrider.*|.*perplexity.*|.*google.*bard.*|.*bard.*google.*|.*bard.*|.*edgeservices.*|.*astastic.*|.*copy\.ai.*|.*bnngpt.*|.*gemini.*google.*$"

def get_creds_smart():
    if 'creds' in globals() and globals()['creds'] and globals()['creds'].valid:
        return globals()['creds']
    try:
        return authenticate_analytics()
    except NameError:
        print("⚠️ Bitte Block 1 (Auth) zuerst ausführen.")
        return None

def classify_traffic_source(source, medium):
    s = str(source).lower()
    m = str(medium).lower()
    if re.match(LLM_REGEX, s): return "LLM / AI"
    if 'organic' in m: return "Organic Search"
    return "Other"

def get_conversion_data_safe(service, property_id):
    """Holt Daten mit Fallback."""
    metrics_try = [{"name": "sessions"}, {"name": "keyEvents"}]
    req = {
        "dateRanges": [{"startDate": START_DATE, "endDate": END_DATE}],
        "dimensions": [{"name": "yearMonth"}, {"name": "sessionSource"}, {"name": "sessionMedium"}],
        "metrics": metrics_try
    }
    try:
        return service.properties().runReport(property=f"properties/{property_id}", body=req).execute()
    except Exception as e:
        if "keyEvents" in str(e) or "400" in str(e):
            req['metrics'] = [{"name": "sessions"}, {"name": "conversions"}]
            try:
                return service.properties().runReport(property=f"properties/{property_id}", body=req).execute()
            except: return None
        return None

def run_median_cvr_analysis():
    creds = get_creds_smart()
    if not creds: return

    print(f"\n🚀 Berechne MEDIAN Conversion Rates (Property-Level)...")

    # Setup
    try:
        admin_service = build('analyticsadmin', 'v1beta', credentials=creds)
        data_service = build('analyticsdata', 'v1beta', credentials=creds)
        acc_summary = admin_service.accountSummaries().list().execute()
        all_props = [p['property'].split('/')[1] for acc in acc_summary.get('accountSummaries', []) for p in acc.get('propertySummaries', [])]
        print(f"👉 Analysiere {len(all_props)} Properties...")
    except: return

    # Daten sammeln
    prop_cvrs = []

    for p_id in tqdm(all_props, desc="Lade Daten"):
        res = get_conversion_data_safe(data_service, p_id)
        if not res or 'rows' not in res: continue

        # Lokaler DataFrame für diese Property
        rows = []
        for r in res['rows']:
            rows.append({
                'Month': r['dimensionValues'][0]['value'],
                'Source': r['dimensionValues'][1]['value'],
                'Medium': r['dimensionValues'][2]['value'],
                'Sessions': int(r['metricValues'][0]['value']),
                'Conversions': float(r['metricValues'][1]['value'])
            })

        df = pd.DataFrame(rows)
        if df.empty: continue

        # Klassifizieren
        df['Channel'] = df.apply(lambda x: classify_traffic_source(x['Source'], x['Medium']), axis=1)
        df = df[df['Channel'].isin(['Organic Search', 'LLM / AI'])]

        # Aggregation PRO PROPERTY
        grouped = df.groupby(['Month', 'Channel']).agg({'Sessions': 'sum', 'Conversions': 'sum'}).reset_index()

        # Filter: Nur Monate mit signifikantem Traffic berücksichtigen (kein Rauschen)
        grouped = grouped[grouped['Sessions'] >= MIN_SESSIONS]

        # CVR berechnen
        grouped['CVR'] = (grouped['Conversions'] / grouped['Sessions']) * 100

        for _, row in grouped.iterrows():
            prop_cvrs.append({
                'Month': row['Month'],
                'Channel': row['Channel'],
                'CVR': row['CVR'],
                'PropertyID': p_id
            })

        time.sleep(0.05)

    if not prop_cvrs:
        print("❌ Keine Daten gefunden.")
        return

    # --- MEDIAN BERECHNUNG ---
    full_df = pd.DataFrame(prop_cvrs)

    # Wir gruppieren nach Monat und Kanal und nehmen den MEDIAN der CVRs aller Properties
    median_trend = full_df.groupby(['Month', 'Channel'])['CVR'].median().reset_index()

    # Pivot für Tabelle
    pivot = median_trend.pivot(index='Month', columns='Channel', values='CVR').fillna(0)

    print("\n" + "="*60)
    print("💰 MEDIAN CVR (%) - TYPISCHE PROPERTY PERFORMANCE")
    print("="*60)
    print("Interpretation: '1.5%' bedeutet, dass 50% deiner Webseiten")
    print("eine bessere und 50% eine schlechtere Conversion Rate haben.")
    print("-" * 60)
    print(pivot.round(2).to_markdown())

    # --- PLOTTING ---
    plt.figure(figsize=(12, 6))

    sns.lineplot(data=median_trend[median_trend['Channel']=='Organic Search'], x='Month', y='CVR',
                 color='gray', linewidth=2, label='Organic Search (Median)')

    sns.lineplot(data=median_trend[median_trend['Channel']=='LLM / AI'], x='Month', y='CVR',
                 color='#a64eff', linewidth=4, marker='o', markersize=8, label='LLM / AI (Median)')

    plt.title('Performance Vergleich: Median Conversion Rate (Portfolio)', fontsize=15)
    plt.ylabel('Median Conversion Rate (%)', fontsize=12)
    plt.xlabel('Monat')
    plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.2)
    plt.legend()
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    run_median_cvr_analysis()

In [ ]:
# @title 💎 RPS Analyse (Streng: Jeden Monat 1+ Purchase, Weighted Avg)
# ==============================================================================
# Filter: Nur Shops, die in JEDEM der 12 Monate mind. 1 Purchase hatten.
# Aggregation: Gewichteter Durchschnitt (Summe Revenue / Summe Sessions).
# Farbe LLM: #a64eff
# ==============================================================================

import re
import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from tqdm import tqdm
from googleapiclient.discovery import build

# --- KONFIGURATION ---
START_DATE = "2025-01-01"
END_DATE   = "2025-12-31"
MIN_SESSIONS_MONTHLY = 10

LLM_REGEX = r"^.*ai|.*\.openai.*|.*copilot.*|.*chatgpt.*|.*gemini.*|.*gpt.*|.*neeva.*|.*writesonic.*|.*nimble.*|.*outrider.*|.*perplexity.*|.*google.*bard.*|.*bard.*google.*|.*bard.*|.*edgeservices.*|.*astastic.*|.*copy\.ai.*|.*bnngpt.*|.*gemini.*google.*$"

def get_creds_smart():
    if 'creds' in globals() and globals()['creds'] and globals()['creds'].valid:
        return globals()['creds']
    try:
        return authenticate_analytics()
    except NameError:
        print("⚠️ Bitte Block 1 (Auth) zuerst ausführen.")
        return None

def classify_traffic_source(source, medium):
    s = str(source).lower()
    m = str(medium).lower()
    if re.match(LLM_REGEX, s): return "LLM / AI"
    if 'organic' in m: return "Organic Search"
    return "Other"

def get_ecommerce_data(service, property_id):
    req = {
        "dateRanges": [{"startDate": START_DATE, "endDate": END_DATE}],
        "dimensions": [{"name": "yearMonth"}, {"name": "sessionSource"}, {"name": "sessionMedium"}],
        "metrics": [{"name": "sessions"}, {"name": "ecommercePurchases"}, {"name": "purchaseRevenue"}]
    }
    try:
        return service.properties().runReport(property=f"properties/{property_id}", body=req).execute()
    except: return None

def check_monthly_activity(df):
    """Prüft, ob in JEDEM Monat Conversions stattfanden."""
    # Wir gruppieren alle Monate, in denen Purchases > 0 waren
    active_months = df[df['Purchases'] > 0]['Month'].unique()
    # Da wir 2025 analysieren, erwarten wir idealerweise 12 Monate (oder so viele wie API liefert)
    # Wir setzen die Hürde auf >= 10 aktive Monate, um kleine Datenlücken zu verzeihen,
    # oder strikt auf 12. Hier: Strikt auf die Anzahl der Monate, die im Datensatz sind.
    return len(active_months) >= 12

def run_strict_rps_analysis():
    creds = get_creds_smart()
    if not creds: return

    print(f"\n🚀 Berechne Weighted RPS (Nur Shops mit Sales in JEDEM Monat)...")

    # Init
    try:
        admin_service = build('analyticsadmin', 'v1beta', credentials=creds)
        data_service = build('analyticsdata', 'v1beta', credentials=creds)
        acc_summary = admin_service.accountSummaries().list().execute()
        all_props = [{'id': p['property'].split('/')[1], 'name': p.get('displayName')}
                     for acc in acc_summary.get('accountSummaries', [])
                     for p in acc.get('propertySummaries', [])]
    except: return

    valid_shops_data = []
    qualified_props_count = 0

    for prop in tqdm(all_props, desc="Filtere Shops"):
        res = get_ecommerce_data(data_service, prop['id'])
        if not res or 'rows' not in res: continue

        # Raw Data Parsing
        rows = []
        for r in res['rows']:
            rows.append({
                'Month': r['dimensionValues'][0]['value'],
                'Source': r['dimensionValues'][1]['value'],
                'Medium': r['dimensionValues'][2]['value'],
                'Sessions': int(r['metricValues'][0]['value']),
                'Purchases': int(r['metricValues'][1]['value']),
                'Revenue': float(r['metricValues'][2]['value'])
            })

        df_raw = pd.DataFrame(rows)
        if df_raw.empty: continue

        # --- FILTER: JEDEN MONAT SALES? ---
        # Wir summieren erst alle Käufe pro Monat (egal welcher Kanal)
        monthly_total = df_raw.groupby('Month')['Purchases'].sum().reset_index()

        # Prüfung: Gibt es 12 Monate mit Purchases > 0?
        # (Wir nehmen an, das Jahr hat 12 Monate im Report)
        months_with_sales = monthly_total[monthly_total['Purchases'] > 0]

        if len(months_with_sales) < 12:
            continue # Property rauswerfen

        qualified_props_count += 1

        # Klassifizierung
        df_raw['Channel'] = df_raw.apply(lambda x: classify_traffic_source(x['Source'], x['Medium']), axis=1)
        df_relevant = df_raw[df_raw['Channel'].isin(['Organic Search', 'LLM / AI'])]

        if df_relevant.empty: continue

        valid_shops_data.append(df_relevant)

    if not valid_shops_data:
        print("❌ Keine Property gefunden, die in JEDEM Monat Sales hatte.")
        return

    print(f"\n✅ {qualified_props_count} 'Dauerläufer-Shops' identifiziert und aggregiert.")

    # --- AGGREGATION (WEIGHTED AVERAGE) ---
    full_df = pd.concat(valid_shops_data)

    # Wir summieren Revenue und Sessions über ALLE validen Shops
    portfolio_agg = full_df.groupby(['Month', 'Channel']).agg({
        'Revenue': 'sum',
        'Sessions': 'sum'
    }).reset_index()

    # Berechnung des gewichteten Durchschnitts
    portfolio_agg['RPS'] = portfolio_agg['Revenue'] / portfolio_agg['Sessions']

    # Tabelle pivotieren
    pivot = portfolio_agg.pivot(index='Month', columns='Channel', values='RPS').fillna(0)

    print("\n" + "="*60)
    print("💎 WEIGHTED REVENUE PER SESSION (Portfolio-Durchschnitt)")
    print("="*60)
    print(pivot.round(2).to_markdown())

    # --- PLOTTING ---
    plt.figure(figsize=(12, 6))

    # Organic
    sns.lineplot(data=portfolio_agg[portfolio_agg['Channel']=='Organic Search'], x='Month', y='RPS',
                 color='gray', linewidth=2, label='Organic Search (Weighted Avg)')

    # LLM
    sns.lineplot(data=portfolio_agg[portfolio_agg['Channel']=='LLM / AI'], x='Month', y='RPS',
                 color='#a64eff', linewidth=4, marker='o', markersize=8, label='LLM / AI (Weighted Avg)')

    plt.title(f'Value per Session 2025: "Dauerläufer" Portfolio (n={qualified_props_count})', fontsize=15)
    plt.ylabel('Revenue / Session (Ø)', fontsize=12)
    plt.xlabel('Monat')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.2)
    plt.legend()
    plt.tight_layout()
    plt.show()

    # --- OPTIONAL: Einzelplots der qualifizierten Shops ---
    print("\n" + "="*60)
    print("🔎 EINZEL-ANALYSEN (Nur qualifizierte Shops)")
    print("="*60)

    # Wir müssen die Namen noch zuordnen, da wir oben DF gesammelt haben
    # (Einfacher Fix: Wir plotten direkt hier nicht mehr, da der User nach "hier nicht als median" gefragt hat
    # und der Fokus auf dem Aggregat liegt. Wenn Einzelplots gewünscht, Code von vorher nutzen.)

if __name__ == "__main__":
    run_strict_rps_analysis()

In [ ]:
# @title 🛍️ 5-Kanal Value Analyse (Active Shops Only)
# ==============================================================================
# Filter: Nur Shops mit Sales in JEDEM Monat 2025.
# Metrik: Weighted Revenue per Session (Absoluter Wert).
# Kanäle: LLM, Ads, Social, Organic, Direct.
# ==============================================================================

import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import re
from tqdm import tqdm
from googleapiclient.discovery import build

# --- KONFIGURATION ---
START_DATE = "2025-01-01"
END_DATE   = "2025-12-31"

LLM_REGEX = r"^.*ai|.*\.openai.*|.*copilot.*|.*chatgpt.*|.*gemini.*|.*gpt.*|.*neeva.*|.*writesonic.*|.*nimble.*|.*outrider.*|.*perplexity.*|.*google.*bard.*|.*bard.*google.*|.*bard.*|.*edgeservices.*|.*astastic.*|.*copy\.ai.*|.*bnngpt.*|.*gemini.*google.*$"
SOCIAL_SOURCES = ['facebook', 'instagram', 'linkedin', 'twitter', 't.co', 'tiktok', 'pinterest', 'reddit', 'youtube']

def get_creds_smart():
    if 'creds' in globals() and globals()['creds'] and globals()['creds'].valid:
        return globals()['creds']
    try:
        return authenticate_analytics()
    except NameError:
        print("⚠️ Bitte Block 1 (Auth) zuerst ausführen.")
        return None

def classify_5_channels(source, medium):
    s = str(source).lower()
    m = str(medium).lower()

    # 1. LLM
    if re.match(LLM_REGEX, s): return "LLM / AI"
    # 2. Ads
    if 'google' in s and ('cpc' in m or 'ppc' in m or 'paid' in m): return "Google Ads"
    # 3. Direct
    if '(direct)' in s: return "Direct"
    # 4. Social
    if 'social' in m or any(soc in s for soc in SOCIAL_SOURCES): return "Social"
    # 5. Organic
    if 'organic' in m: return "Organic Search"

    return "Other"

def get_ecommerce_data(service, property_id):
    req = {
        "dateRanges": [{"startDate": START_DATE, "endDate": END_DATE}],
        "dimensions": [{"name": "yearMonth"}, {"name": "sessionSource"}, {"name": "sessionMedium"}],
        "metrics": [{"name": "sessions"}, {"name": "ecommercePurchases"}, {"name": "purchaseRevenue"}]
    }
    try:
        return service.properties().runReport(property=f"properties/{property_id}", body=req).execute()
    except: return None

def run_5_channel_shop_analysis():
    creds = get_creds_smart()
    if not creds: return

    print(f"\n🚀 Starte 5-Kanal Value Analyse (Nur aktive Shops)...")

    # Init
    try:
        admin_service = build('analyticsadmin', 'v1beta', credentials=creds)
        data_service = build('analyticsdata', 'v1beta', credentials=creds)
        acc_summary = admin_service.accountSummaries().list().execute()
        all_props = [{'id': p['property'].split('/')[1], 'name': p.get('displayName')}
                     for acc in acc_summary.get('accountSummaries', [])
                     for p in acc.get('propertySummaries', [])]
    except: return

    valid_shops_data = []
    qualified_props_count = 0

    # Daten Loop
    for prop in tqdm(all_props, desc="Filtere Shops"):
        res = get_ecommerce_data(data_service, prop['id'])
        if not res or 'rows' not in res: continue

        rows = []
        for r in res['rows']:
            rows.append({
                'MonthStr': r['dimensionValues'][0]['value'],
                'Source': r['dimensionValues'][1]['value'],
                'Medium': r['dimensionValues'][2]['value'],
                'Sessions': int(r['metricValues'][0]['value']),
                'Purchases': int(r['metricValues'][1]['value']),
                'Revenue': float(r['metricValues'][2]['value'])
            })

        df_raw = pd.DataFrame(rows)
        if df_raw.empty: continue

        # FILTER: Dauerläufer-Check (Sales in jedem Monat)
        monthly_check = df_raw.groupby('MonthStr')['Purchases'].sum()
        if (monthly_check > 0).sum() < 12:
            continue

        qualified_props_count += 1

        # Klassifizierung
        df_raw['Channel'] = df_raw.apply(lambda x: classify_5_channels(x['Source'], x['Medium']), axis=1)
        # Nur relevante Kanäle
        df_relevant = df_raw[df_raw['Channel'] != 'Other']

        if df_relevant.empty: continue
        valid_shops_data.append(df_relevant)

    if not valid_shops_data:
        print("❌ Keine aktiven Shops gefunden.")
        return

    print(f"✅ {qualified_props_count} Shops qualifiziert.")

    # --- AGGREGATION ---
    full_df = pd.concat(valid_shops_data)
    full_df['Date'] = pd.to_datetime(full_df['MonthStr'], format='%Y%m')

    # Weighted Average Berechnung
    portfolio_agg = full_df.groupby(['Date', 'Channel']).agg({
        'Revenue': 'sum',
        'Sessions': 'sum'
    }).reset_index()

    portfolio_agg['RPS'] = portfolio_agg['Revenue'] / portfolio_agg['Sessions']
    portfolio_agg = portfolio_agg.sort_values('Date')

    # --- TABELLE ---
    portfolio_agg['Monat'] = portfolio_agg['Date'].dt.strftime('%Y-%m')
    pivot = portfolio_agg.pivot(index='Monat', columns='Channel', values='RPS').fillna(0)

    # Sortierung für Tabelle
    desired_order = ['LLM / AI', 'Google Ads', 'Social', 'Organic Search', 'Direct']
    cols = [c for c in desired_order if c in pivot.columns]
    pivot = pivot[cols]

    print("\n" + "="*60)
    print("💰 VALUE PER SESSION (Absolute Währung, Weighted Avg)")
    print("="*60)
    print(pivot.round(2).to_markdown())

    # --- PLOTTING ---
    plt.figure(figsize=(14, 8))

    colors = {
        'LLM / AI': '#a64eff',       # Violett
        'Google Ads': '#DB4437',     # Rot
        'Social': '#4285F4',         # Blau
        'Organic Search': '#0F9D58', # Grün
        'Direct': '#757575'          # Grau
    }

    for channel in cols:
        subset = portfolio_agg[portfolio_agg['Channel'] == channel]
        if subset.empty: continue

        # LLM hervorheben
        lw = 4 if channel == 'LLM / AI' else 2
        style = '--' if channel == 'LLM / AI' else '-'
        alpha = 1.0 if channel == 'LLM / AI' else 0.75

        sns.lineplot(data=subset, x='Date', y='RPS',
                     label=channel, color=colors.get(channel, 'black'),
                     linewidth=lw, linestyle=style, marker='o', markersize=6, alpha=alpha)

        # Label am Ende
        last = subset.iloc[-1]
        plt.text(last['Date'], last['RPS'], f" {last['RPS']:.2f}",
                 color=colors.get(channel, 'black'), fontweight='bold', va='center')

    plt.title(f'Value Comparison 2025: Welcher Kanal bringt den wertvollsten Traffic?\n(Basis: {qualified_props_count} Dauerläufer-Shops)', fontsize=16)
    plt.ylabel('Revenue per Session (Ø)', fontsize=12)
    plt.xlabel('')

    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.gcf().autofmt_xdate()

    plt.grid(True, alpha=0.2)
    plt.legend()
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    run_5_channel_shop_analysis()